In [1]:
from discretize import TensorMesh
from SimPEG import maps
from simpegEM1D.simulation_original import EM1DFMSimulation
from simpegEM1D.survey_original import EM1DSurveyFD
from simpegEM1D import analytics
import numpy as np
import matplotlib
%pylab inline

Populating the interactive namespace from numpy and matplotlib


# Set up for EM1D program

## Step1: Generate mesh

In [2]:
nearthick = np.logspace(-1, 1, 5)
deepthick = np.logspace(1, 2, 10)
hx = np.r_[nearthick, deepthick]
mesh1D = TensorMesh([hx], [0.])
depth = -mesh1D.gridN[:-1]
LocSigZ = -mesh1D.gridCC
nlay = depth.size
topo = np.r_[0., 0., 0.]

## Step2: Set survey parameters

In [3]:
FDsurvey = EM1DSurveyFD(
    rx_location = np.array([0., 0., 0.]),
    src_location = np.array([0., 0., 0.]),
    field_type = 'secondary',
    rx_type = 'Hz',
    src_type = 'VMD',
    depth = depth,
    topo = topo,
    frequency = np.logspace(-2, 8, 61),
    offset = 8. * np.ones(61),
    a = 1.,
    I = 1.
)

Colesurvey = EM1DSurveyFD(
    rx_location = np.array([0., 0., 0.]),
    src_location = np.array([0., 0., 0.]),
    field_type = 'secondary',
    rx_type = 'Hz',
    src_type = 'VMD',
    depth = depth,
    topo = topo,
    frequency = np.logspace(-2, 8, 61),
    offset = 8. * np.ones(61),
    a = 1.,
    I = 1.
)

print(np.shape(FDsurvey.frequency))
print(FDsurvey.frequency)

(61,)
[1.00000000e-02 1.46779927e-02 2.15443469e-02 3.16227766e-02
 4.64158883e-02 6.81292069e-02 1.00000000e-01 1.46779927e-01
 2.15443469e-01 3.16227766e-01 4.64158883e-01 6.81292069e-01
 1.00000000e+00 1.46779927e+00 2.15443469e+00 3.16227766e+00
 4.64158883e+00 6.81292069e+00 1.00000000e+01 1.46779927e+01
 2.15443469e+01 3.16227766e+01 4.64158883e+01 6.81292069e+01
 1.00000000e+02 1.46779927e+02 2.15443469e+02 3.16227766e+02
 4.64158883e+02 6.81292069e+02 1.00000000e+03 1.46779927e+03
 2.15443469e+03 3.16227766e+03 4.64158883e+03 6.81292069e+03
 1.00000000e+04 1.46779927e+04 2.15443469e+04 3.16227766e+04
 4.64158883e+04 6.81292069e+04 1.00000000e+05 1.46779927e+05
 2.15443469e+05 3.16227766e+05 4.64158883e+05 6.81292069e+05
 1.00000000e+06 1.46779927e+06 2.15443469e+06 3.16227766e+06
 4.64158883e+06 6.81292069e+06 1.00000000e+07 1.46779927e+07
 2.15443469e+07 3.16227766e+07 4.64158883e+07 6.81292069e+07
 1.00000000e+08]


## Step3: Set mappings (Exponential and Cole-Cole maps)

### Exponential map

### $$ m = log(\sigma) $$

### $$ M_{exp}(m) = e^{m}$$

### Cole-Cole map

$$ m(\sigma_{\infty}, \eta, \tau, c; \omega) = \sigma_{\infty} - \sigma_{\infty}\eta \frac{1}{1+(1-\eta)(\imath\omega\tau)^c}$$

In [4]:
sig_half = 1e-2
chi_half = 0.
Expmap = maps.ExpMap(mesh1D)
tau =  np.ones(nlay, dtype=float)*1e-1
eta = np.ones(nlay, dtype=float)*0.5
c = np.ones(nlay, dtype=float)*0.88

In [5]:
m_1D = np.log(np.ones(nlay)*sig_half)

In [6]:
# Conductivity
simFD = EM1DFMSimulation(
    mesh1D, survey=FDsurvey, sigmaMap=Expmap,
    chi=np.zeros(nlay)
)

# Hz = simFD.dpred(m_1D)
Hz = simFD.fields(m_1D)
# print(np.shape(Hz))
# print(Hz)

AttributeError: 'EM1DFMSimulation' object has no attribute 'half_switch'

In [ ]:
# Frequency-dependent conductivity
simCole = EM1DFMSimulation(
    mesh1D, survey=Colesurvey, sigmaMap=Expmap,
    eta=eta, tau=tau, c=c,
    chi=np.zeros(nlay)
)

# HzCole = simCole.dpred(m_1D)
HzCole = simCole.fields(m_1D)
Hzanal = analytics.Hzanal(sig_half, FDsurvey.frequency, FDsurvey.offset, 'secondary')
print(np.shape(HzCole))
print(HzCole)

In [ ]:
matplotlib.rcParams['font.size'] = 16

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 5))
ax.loglog(FDsurvey.frequency, np.abs(np.real(Hz)), 'b-')
ax.loglog(FDsurvey.frequency, np.abs(np.real(Hzanal)), 'bx')
ax.loglog(FDsurvey.frequency, np.abs(np.imag(Hz)), 'r-')
ax.loglog(FDsurvey.frequency, np.abs(np.imag(Hzanal)), 'rx')
legend(('Real $H_{z}$', 'Real $H_{z \ true}$', 'Imag $H_{z}$', 'Imag $H_{z \ true}$'))

In [ ]:
fig, ax = plt.subplots(1,1, figsize = (8, 5))
ax.loglog(FDsurvey.frequency, np.abs(np.real(HzCole)), 'b-')
ax.loglog(FDsurvey.frequency, np.abs(np.real(Hzanal)), 'bx')
ax.loglog(FDsurvey.frequency, np.abs(np.imag(HzCole)), 'r-')
ax.loglog(FDsurvey.frequency, np.abs(np.imag(Hzanal)), 'rx')
legend(('Real $H_{z}$', 'Real $H_{z \ IP}$', 'Imag $H_{z}$', 'Imag $H_{z \ IP}$'))